# Red Neuronal Profunda (DNN) para clasificación MNIST

Aplicaremos todos nuestros conocimientos para crear una DNN, frecuentemente llamada también una Artificial Neural Network (ANN).  El problema que vamos a trabajar se conoce como el "Hola Mundo" del aprendizaje profundo porque para la mayoría de estudiantes este es el primer algoritmo de aprendizaje profundo que ven. 

El conjunto de datos se llama MNIST y se refiere al reconocimiento de dígitos escritos a mano.  Pueden encontrar más información en el sitio web de Yann LeCun (Director of AI Research, Facebook).  El es uno de los pioneros de todo este tema, así como de otras metodologías más complejas como las Redes Neurales Convolucionales (CNN) que se utilizan hoy día.

El conjunto de datos tiene 70,000 imágenes (28x28 pixels) de dígitos escritos a mano (1 dígito por imagen).

La meta es escribir un algoritmo que detecta qué dígito ha sido escrito.  Como solo hay 10 dígitos (0 al 9), este es un problema de clasificación con 10 clases.

Nuestra meta será construir una RN con 2 capas escondidas.

## Plan de Acción para preparar el modelo

1.  Preparar los datos y preprocesarlos.  Crear los conjuntos de datos para entrenar, validar y probar
2.  Crear un esboso del modelo y seleccionar las funciones de activación
3.  Fijar los optimizadores avanzados y la función de pérdida
4.  Hacer que el modelo aprenda
5.  Probar la exactitud ("accuracy") del modelo

## Importar los paquetes relevantes

TensorFlow incluye un proveedor de los datos de MNIST que utilizaremos acá.  Viene con el módulo **"tensorflow.keras.datasets"**. 

In [1]:
import numpy as np
import tensorflow as tf

La siguiente instrucción, cuando se corre por primera vez, descarga el conjunto de datos en lo indicado por el parámetro path, relativo a  ~/.keras/datasets).  Como si se hubiera ejecutado Lo siguiente:

tf.keras.datasets.mnist.load_data(
    path = 'mnist.npz'
)

luego separa los datos en un conjunto para entrenamiento y otro para pruebas.

Si se ejecuta más de una vez, ya no descarga el archivo.

In [2]:
(X_entreno, y_entreno), (X_prueba, y_prueba) = tf.keras.datasets.mnist.load_data()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
X_entreno

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

Como no podemos ver la forma de los conjuntos...les queda de tarea averiguar por qué no...podemos utilizar la instrucción **assert**

In [4]:
assert X_entreno.shape == (60000, 28, 28)
assert X_prueba.shape == (10000, 28, 28)
assert y_entreno.shape == (60000,)
assert y_prueba.shape == (10000,)

## Datos

Esta sección es donde pre-procesaremos nuestros datos.

Por default, TF2 tiene conjuntos de datos de entrenamiento y de prueba, pero no tiene un conjunto de validación, por lo que debemos dividirlo por nuestra cuenta

Lo haremos del mismo tamaño que el conjunto de prueba

In [5]:
num_obs_validacion = y_prueba.shape[0]

Usaremos una variable dedicada para el número de observaciones de prueba

In [6]:
num_obs_prueba = y_prueba.shape[0]

Generalmente preferimos "normalizar" nuestros datos en alguna forma para que el resultado sea numéricamente más estable.  En este caso simplemente preferimos tener entradas entre 0 y 1, por lo que definimos una función, que reciba la imagen MNIST.

Como los posibles valores de las entradas son entre 0 y 255 (256 posibles tonos de gris), al dividirlos por 255 obtenemos el resultado deseado.

In [7]:
X_entreno_normalizado = X_entreno / 255

Finalmente, normalizaremos y convertiremos los datos de pruebas en tandas.  Los normalizamos para que tengan la misma magnitud que los datos de entrenamiento y validación.

No hay necesidad de "barajearlo" ya que no estaremos entrenando con los datos de prueba.  Habra una sola tanda, igual al tamaño de los datos de prueba.

In [8]:
X_prueba_normalizado = X_prueba / 255

Una vez se han "normalizado" los datos, podemos proceder a extraer los datos de entrenamiento y de validación.

Nuestros datos de validación serán 10000 para ser igual al conjunto de prueba.

Finalmente, creamos una tanda con un tamaño de tanda igual al total de muestras de validación.

In [9]:
X_validacion = X_entreno_normalizado[-num_obs_validacion: , : , : ]
y_validacion = y_entreno[-num_obs_validacion:]

Similarmente, los datos de entrenamiento son todos los demás por lo que nos salteamos tantas observaciones como las hay en el conjunto de validación.

In [10]:
X_entreno = X_entreno_normalizado[ : X_entreno_normalizado.shape[0] - num_obs_validacion, : , : ]
y_entreno = y_entreno[ : y_entreno.shape[0] - num_obs_validacion]
num_obs_entreno = y_entreno.shape[0]

Convertir de Arreglos Numpy a Tensores

In [11]:
datos_entreno = tf.data.Dataset.from_tensor_slices((X_entreno, y_entreno))
datos_validacion = tf.data.Dataset.from_tensor_slices((X_validacion, y_validacion))
datos_prueba = tf.data.Dataset.from_tensor_slices((X_prueba, y_prueba))

Barajear y hacer tandas con el conjunto de datos de entrenamiento

In [12]:
TAMANIO_TANDA = 100
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)

Hacer tandas con los conjuntos de validación y prueba, no se necesita barajearlos

In [13]:
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

## Modelo

### Delineamos el modelo

Cuando pensamos sobre un algoritmo de aprendizaje profundo, casi siempre imaginamos la realización del mismo.  Asi que esta vez, hagámoslo.  :)

In [14]:
tamanio_entrada = 784
tamanio_salida = 10

Usaremos el mismo ancho para ambas capas escondidas.  (No es una necesidad!)

In [15]:
tamanio_capa_escondida = 50

# Definimos cómo se verá el modelo

La primera capa (la de entrada):  cada observación es de 28x28 píxeles, por lo tanto es un tensor de rango 2.

Como aún no hemos aprendido sobre CNNs, no sabemos como alimentar este tipo de entrada a nuestra red, por lo tanto hay que "aplanar" las imágenes.  Hay un método conveniente **Flatten** que toma nuestro tensor de 28x28 y lo convierte en  un vector (None,) o (784,)...porque 28x28 = 784.  Esto nos permite crear una red de alimentación hacia adelante.

    
**tf.keras.layers.Dense** básicamente implementa:  *salida = activation(dot(entrada, peso) + sesgo)*.  Requiere varios argumentos, pero los más importantes para nosotros son el ancho de la capa escondida y la función de activación.

La capa final no es diferente, solo nos aseguramos de activarla con **softmax**


In [16]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

c:\Users\lpmon\Documents\GitHub\Lab2-DataScience\venv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### Seleccionar el optimizador y la función de pérdida

Definimos el optimizador que nos gustaría utilizar, la función de pérdida, y las métricas que nos interesa obtener en cada interacción

In [17]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Entrenamiento

Acá es donde entrenamos el modelo que hemos construído

Determinamos el número máximo de épocas.

Ajustamos el modelo , especificando:

* los datos de entrenamiento
* el número total de épocas
* y los datos de validación que creamos en el formato (entradas, metas)

In [18]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

Epoch 1/5
500/500 - 1s - 2ms/step - accuracy: 0.8760 - loss: 0.4346 - val_accuracy: 0.9426 - val_loss: 0.2138
Epoch 2/5
500/500 - 1s - 1ms/step - accuracy: 0.9410 - loss: 0.2003 - val_accuracy: 0.9556 - val_loss: 0.1614
Epoch 3/5
500/500 - 0s - 981us/step - accuracy: 0.9544 - loss: 0.1527 - val_accuracy: 0.9613 - val_loss: 0.1363
Epoch 4/5
500/500 - 0s - 964us/step - accuracy: 0.9623 - loss: 0.1252 - val_accuracy: 0.9607 - val_loss: 0.1352
Epoch 5/5
500/500 - 0s - 990us/step - accuracy: 0.9689 - loss: 0.1048 - val_accuracy: 0.9670 - val_loss: 0.1143


## Probar el modelo

Como se discutió en clase, luego del entrenamiento (con los datos de entrenamiento), y la validación (con los datos de validación), probamos el potencial de predicción final de nuestro modelo con el conjunto de datos de prueba que el algoritmo NUNCA ha visto antes.

Es muy importante reconocer que estar "jugando" con los hiperparámetros sobre-ajusta el conjunto de datos de validación.

La prueba es la instancia absolutamente final. **NUNCA** debe probarse el modelo antes de haber completamente ajustado el mismo.

Si se ajusta el modelo después de hacer la prueba, se empezará a sobre-ajustar el conjunto de datos de prueba, que echaría "por los suelos" el propósito original del mismo.

In [19]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - accuracy: 0.9622 - loss: 19.0089


In [20]:
# Si se desea, se puede aplicar un formateo "bonito"
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Pérdida de prueba: 19.01. Precisión de prueba: 96.22%


Utilizando el modelo inicial y los hiperparámetros dados en este notebook, la precisión de prueba final debe ser aproximadamente 97%.

Cada vez que se ejecuta el código, se obtiene una precisión diferente debido a la "barajeada" de las tandas, los pesos se inicializan en forma diferente, etc.

Finalmente, intencionalmente se ha llegado a una solución subóptima, para que puedan tener la oportunidad de mejorarla como ejercicio de laboratorio.

## Nuevo Modelo
En este modelo se hará los siguientes cambios:
- Modificación de ancho de red
- Modificación de profundidad 
- Experimentación con redes profundas
- Funciones de activación
- Modificación de la tanda

### Modificación del ancho de red
Se modifica el tamaño de capa escondida a 200 y se vuelve a entrenar

In [75]:
tamanio_capa_escondida = 300 # modify hidden layer size 

In [76]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

In [77]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [78]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

Epoch 1/5
500/500 - 2s - 3ms/step - accuracy: 0.9235 - loss: 0.2600 - val_accuracy: 0.9649 - val_loss: 0.1171
Epoch 2/5
500/500 - 1s - 2ms/step - accuracy: 0.9703 - loss: 0.0978 - val_accuracy: 0.9698 - val_loss: 0.1034
Epoch 3/5
500/500 - 1s - 2ms/step - accuracy: 0.9806 - loss: 0.0632 - val_accuracy: 0.9776 - val_loss: 0.0776
Epoch 4/5
500/500 - 1s - 2ms/step - accuracy: 0.9859 - loss: 0.0438 - val_accuracy: 0.9745 - val_loss: 0.0882
Epoch 5/5
500/500 - 1s - 2ms/step - accuracy: 0.9891 - loss: 0.0334 - val_accuracy: 0.9759 - val_loss: 0.0889


In [79]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9774 - loss: 13.5997  


In [80]:
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Pérdida de prueba: 13.60. Precisión de prueba: 97.74%


### Resultados
1. ¿Cómo cambia la precisión de validación del modelo?<br>
En este caso tuvimos menor pérdida con 200 que con el valor anterior de 50. La precisión fue un poco más elevada 
2. ¿Cuánto tiempo tarda el algoritmo en entrenar?<br>
Con 200 capas escondidas tardó bastante poco. Unos cuantos milisegundos.
3. ¿Cuál ofrece mejor rendimiento?<br>
Con la de 200 tuvimos pérdida baja, similarmente con 100 hubo unos resultados similares. Ya cuando intentamos utilizar 300<br>
la cantidad de pérdida fue aumentando aunque la precisión fue similar. Mientras que con 500 capas, la pérdida fue ligeramente menor pero aumentó por igual
y con una precisión de casi 98%. <br>
Por lo que podemos ver que realmente el rendimiento se mantiene bastante bueno, pero entre más capas, hay menor pérdida, pero más tiempo de entrenamiento.
Por lo que vimos que en rendimiento y precisión era menor 300 capas. 

## Modificación de la profundiad de la red
En este caso le agregaremos una nueva capa oculta al modelo.<br>
De momento los valores a utilizar son:
* tamaño de capa : 300
* tamaño entrada : 784
* tamaño salida : 10<br>
Se utiliza<br>
* activación ReLU para cada capa escondida
* activación softmax para la de salida
* optimizador adam
* pérdida con entropía cruzada
* 5 épocas 

In [81]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3ra capa escondida agregada

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

c:\Users\lpmon\Documents\GitHub\Lab2-DataScience\venv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [82]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [83]:
modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

Epoch 1/5
500/500 - 2s - 4ms/step - accuracy: 0.9236 - loss: 0.2533 - val_accuracy: 0.9616 - val_loss: 0.1275
Epoch 2/5
500/500 - 1s - 3ms/step - accuracy: 0.9714 - loss: 0.0938 - val_accuracy: 0.9722 - val_loss: 0.0936
Epoch 3/5
500/500 - 1s - 3ms/step - accuracy: 0.9803 - loss: 0.0626 - val_accuracy: 0.9732 - val_loss: 0.0924
Epoch 4/5
500/500 - 1s - 3ms/step - accuracy: 0.9842 - loss: 0.0467 - val_accuracy: 0.9761 - val_loss: 0.0830
Epoch 5/5
500/500 - 1s - 3ms/step - accuracy: 0.9879 - loss: 0.0375 - val_accuracy: 0.9781 - val_loss: 0.0760


In [84]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9794 - loss: 13.6910


### Resultados
1. Precisión de validación con el anterior modelo,<br>
Este modelo presenta una validación similar a la del modelo anterior. Su exactitud es similar y su pérdida también. 
2. Impacto en tiempo de ejecución.<br>
El tiempo de ejecución se vio levemente afectado por el añadimiento de una capa escondida extra.
3. Cambios necesarios al código.<br>
El código se mantuvi similar, solamente se tuvo que agregar la línea <br>
`tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'),` <br>
al modelo para que agregarse una capa escondida. 

## Redes Profundas
Aquí experimentaremos con el ancho de cada capa y cómo afecta esto en su tiempo de ejecución. <br>
Veremos si existe problemas de desvanecimiento de gradiente. 

In [89]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3ra capa escondida agregada
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 4ta capa escondida agregada 
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 5ta capa escondida agregada 

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

In [90]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [91]:
modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

Epoch 1/5
500/500 - 2s - 5ms/step - accuracy: 0.9216 - loss: 0.2582 - val_accuracy: 0.9635 - val_loss: 0.1265
Epoch 2/5
500/500 - 2s - 3ms/step - accuracy: 0.9689 - loss: 0.1044 - val_accuracy: 0.9668 - val_loss: 0.1050
Epoch 3/5
500/500 - 2s - 3ms/step - accuracy: 0.9773 - loss: 0.0723 - val_accuracy: 0.9716 - val_loss: 0.0969
Epoch 4/5
500/500 - 2s - 3ms/step - accuracy: 0.9813 - loss: 0.0581 - val_accuracy: 0.9709 - val_loss: 0.1076
Epoch 5/5
500/500 - 2s - 3ms/step - accuracy: 0.9857 - loss: 0.0462 - val_accuracy: 0.9770 - val_loss: 0.0853


In [92]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9731 - loss: 17.0010 


### Resultados
1. Precisión de validación para cada intento<br>
    - 4 capas: accuracy 97.54% / pérdida 15.35
    - 5 capas: accuracy 97.31% / pérdida 17.00
2. Relación entre profundidad y tiempo de ejecución<br>
    Como podemos ver, entre más capas el tiempo de ejecución del modelo va demorando poco a poco<br>
    cada vez más. Esto se debe sencillamente a que entre más capas tenemos, hay más transformaciones e interacciones <br>
    entre las capas neuronales. Lo que causa que se demore cada vez más conforme se aumenta el número de capas.
3. Problemas de desvanecimiento de gradiente<br>
    Entre más capas neuronales tenemos, más activaciones estaremos haciendo, lo cual puede llevar a que
    los coeficientes vayan siendo más y más pequeños conforme se agregan más capas debido a que si los <br>
    coeficientes son menores a 1, la multiplicación entre estos dará como resultado un número más y más pequeño<br>
    entre más capas pase. Por eso a veces más capas no significa mejor resultado. 

## Funciones de Activación I
Aplicaremos funciones de activación sigmoidales a todas las capas. 

In [ ]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 3ra capa escondida agregada
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 4ta capa escondida agregada 
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 5ta capa escondida agregada 

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

In [106]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [107]:
modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

Epoch 1/5
500/500 - 2s - 5ms/step - accuracy: 0.7038 - loss: 0.8700 - val_accuracy: 0.9127 - val_loss: 0.3124
Epoch 2/5
500/500 - 2s - 4ms/step - accuracy: 0.9236 - loss: 0.2703 - val_accuracy: 0.9462 - val_loss: 0.1890
Epoch 3/5
500/500 - 2s - 4ms/step - accuracy: 0.9482 - loss: 0.1802 - val_accuracy: 0.9551 - val_loss: 0.1554
Epoch 4/5
500/500 - 2s - 4ms/step - accuracy: 0.9599 - loss: 0.1399 - val_accuracy: 0.9608 - val_loss: 0.1349
Epoch 5/5
500/500 - 2s - 4ms/step - accuracy: 0.9671 - loss: 0.1134 - val_accuracy: 0.9608 - val_loss: 0.1389


In [108]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9536 - loss: 0.1758


### Resultados
1. Compare resultados anteriores
    - 3 capas sigmoiodales : accuracy 96.26% / pérdida 0.12
    - 3 capas ReLU : accuracy 97.94 / pérdida 13.69
    - 4 capas sigmoidales : 95.61 / pérdida 0.1605
    - 4 capas ReLU : accuracy 97.54% / pérdida 15.35
    - 5 capas sigmoidales : 95.36 / pérdida 0.1758
    - 5 capas ReLU : accuracy 97.31% / pérdida 17.00
2. Analice el impacto de velocidad de convergencia<br>
    La velocidad de convergencia es un poco diferente pero en este caso parece ser mejor.

## Funciones de Activación II

In [121]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='tanh'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 3ra capa escondida agregada
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 4ta capa escondida agregada 
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 5ta capa escondida agregada 

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

In [122]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [123]:
modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

Epoch 1/5
500/500 - 2s - 5ms/step - accuracy: 0.8627 - loss: 0.4503 - val_accuracy: 0.9573 - val_loss: 0.1543
Epoch 2/5
500/500 - 2s - 3ms/step - accuracy: 0.9632 - loss: 0.1277 - val_accuracy: 0.9681 - val_loss: 0.1128
Epoch 3/5
500/500 - 2s - 3ms/step - accuracy: 0.9752 - loss: 0.0860 - val_accuracy: 0.9690 - val_loss: 0.1152
Epoch 4/5
500/500 - 2s - 4ms/step - accuracy: 0.9808 - loss: 0.0649 - val_accuracy: 0.9710 - val_loss: 0.1080
Epoch 5/5
500/500 - 2s - 4ms/step - accuracy: 0.9856 - loss: 0.0486 - val_accuracy: 0.9733 - val_loss: 0.1034


In [124]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9736 - loss: 0.1035


### Resultados
1. Comparación entre modelos.
    - 3 capas sigmoiodales : accuracy 96.26% / pérdida 0.12
    - 3 capas ReLU : accuracy 97.47% / pérdida 0.0995
    - 3 capas ReLU & Tanh : accuracy 97.45% / pérdida 0.1220
    - 4 capas sigmoidales : 95.61% / pérdida 0.1605
    - 4 capas ReLU : accuracy 97.54% / pérdida 15.35
    - 4 capas ReLU & Tanh : accuracy 97.40% / pérdida 0.1023
    - 5 capas sigmoidales : 95.36% / pérdida 0.1758
    - 5 capas ReLU : accuracy 97.31% / pérdida 17.00
    - 5 capas ReLU & Tanh : accuracy 97.36% / pérdida 0.1035
2. Las ventajas y desventajas.<br>
    ReLU: <br>
    - ventaja:
    - desventaja: 
    Sigmoidal: <br>
    - ventaja: 
    - desventaja: 
    Tanh: <br>
    - ventaja:
    - desventaja: 
    

## Modificar el tamaño del batch
En este caso vamos a modificar el tamaño del batch a 10000 para ver los cambios de desempeño y pérdida.

In [125]:
TAMANIO_TANDA = 10000
datos_entreno = datos_entreno.shuffle(buffer_size = num_obs_entreno).batch(TAMANIO_TANDA)
datos_validacion = datos_validacion.batch(TAMANIO_TANDA)
datos_prueba = datos_prueba.batch(TAMANIO_TANDA)

In [126]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='tanh'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 3ra capa escondida agregada
    # tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 4ta capa escondida agregada 
    # tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 5ta capa escondida agregada 

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

c:\Users\lpmon\Documents\GitHub\Lab2-DataScience\venv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [127]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [128]:
modelo.fit(datos_entreno,
          epochs = NUMERO_EPOCAS, 
          validation_data = datos_validacion,
          verbose = 2)

Epoch 1/5


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("Cast:0", shape=(None, None, 28, 28), dtype=float32). Expected shape (None, 28, 28), but input has incompatible shape (None, None, 28, 28)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, None, 28, 28), dtype=float32)
  • training=True
  • mask=None
  • kwargs=<class 'inspect._empty'>

In [ ]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)